# LLP Trigger Efficiency Study

This notebook is meant to serve as the development space for code related to the efficiency study of HCAL LLP triggers

## Setup

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import uproot
import mplhep as hep
import os
import awkward as ak
import hist
import itertools
import pandas as pd
import pickle
import re
from pypdf import PdfMerger

hep.style.use("CMS")

In [2]:
import sys
sys.path.append("./src/")
import utils
from utils.LLPTrigUtils import compute_efficiencies, findBestTrig, printBestTrigs
from utils.LLPTrigPlotting import plot_efficiencies, plot_improvements
from utils.TrigUtils import genFileName

In [3]:
def reload_module(module):
    import importlib
    importlib.reload(module)

In [4]:
# HLT LLP Triggers
hltTrigs = [
    'HLT_HT170_L1SingleLLPJet_DisplacedDijet40_DisplacedTrack',
    'HLT_HT200_L1SingleLLPJet_DisplacedDijet40_DisplacedTrack',
    'HLT_HT200_L1SingleLLPJet_DisplacedDijet60_DisplacedTrack',
    'HLT_HT270_L1SingleLLPJet_DisplacedDijet40_DisplacedTrack',
    'HLT_HT320_L1SingleLLPJet_DisplacedDijet60_Inclusive',
    'HLT_HT420_L1SingleLLPJet_DisplacedDijet60_Inclusive',
    'HLT_HT200_L1SingleLLPJet_DelayedJet40_SingleDelay1nsTrackless',
    'HLT_HT200_L1SingleLLPJet_DelayedJet40_SingleDelay2nsInclusive',
    'HLT_HT200_L1SingleLLPJet_DelayedJet40_DoubleDelay0p5nsTrackless',
    'HLT_HT200_L1SingleLLPJet_DelayedJet40_DoubleDelay1nsInclusive',
    'HLT_HT200_L1SingleLLPJet_DisplacedDijet30_Inclusive1PtrkShortSig5',
    'HLT_HT200_L1SingleLLPJet_DisplacedDijet35_Inclusive1PtrkShortSig5',
    'HLT_HT200_L1SingleLLPJet_DisplacedDijet40_Inclusive1PtrkShortSig5'
    ]

# L1 LLP Triggers
l1Trigs = [
    'L1_DoubleLLPJet40',
    'L1_HTT120_SingleLLPJet40',
    'L1_HTT160_SingleLLPJet50',
    'L1_HTT200_SingleLLPJet60',
    'L1_HTT240_SingleLLPJet70'
    ]

## L1 Triggers

In [5]:
# Getting name of unprescaled L1 triggers
prescales = np.genfromtxt(
    fname='/uscms/home/roycruz/nobackup/EMJ/analysis/prescales/Prescale_2022_v0_1_1.csv', 
    delimiter=',', dtype='i8,U128,i8,i8,i8,i8,i8,i8,i8,i8,i8', names=True)
unprescaled = prescales["Name"][prescales["2E34"]==1]

### s-channel

In [ ]:
# Defining parameter values
data_dir = "/eos/uscms/store/user/rcruzcan/EMJsamples/"

# Lists of parameters to use
channel = "s"
mMed_lst= [100, 250, 500, 750, 1000, 1500, 2000]
mDark_lst = [10, 20]
ctau_lst = [1, 100, 1000, 1500, 2000]

genfnames = []
for params in itertools.product(mMed_lst, mDark_lst, ctau_lst):
    genfnames.append(genFileName(params[0], params[1], params[2], channel=channel))
print("Files to be processed")
genfnames

In [ ]:
# Computing efficiencies
entry_stop = -1
trigs = l1Trigs

effs_dict = compute_efficiencies(
    trigs, data_dir, mMed_lst, mDark_lst, ctau_lst, channel,
    mode="L1", 
    entry_stop=entry_stop,
    unprescaled=unprescaled,
)

In [8]:
# Saving the efficiencies in a file for convenience
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienL1LLP_schan_effs.pkl", "wb") as f:
    pickle.dump(effs_dict, f)

In [9]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienL1LLP_schan_effs.pkl", "rb") as f:
    effs_dict = pickle.load(f)

In [ ]:
# Plotting raw efficiencies
savefig = True 
plot_efficiencies(
    effs_dict, mMed_lst, ctau_lst, mDark_lst, l1Trigs, 
    plotbest=False, 
    savefig=savefig, 
    outfname="sChanL1LLP_plots", 
    title="EMJ s-channel LLP L1 Trigger Efficiency ", 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots",
    figsize=(12, 8),
    unitymax=True,
    cleandir=True
)

In [ ]:
# Plotting efficiencies of LLP + best
savefig=True
plot_improvements(
    effs_dict, mMed_lst, ctau_lst, mDark_lst, l1Trigs, 
    plotbest=True, 
    figsize=(12, 8), 
    title="EMJ s-channel LLP L1+Best Trigger Efficiency ", 
    savefig=savefig, 
    outfname="sChanL1LLP_improvements",
    unitymax=True
)

### t-channel

In [ ]:
data_dir = "/eos/uscms/store/user/rcruzcan/EMJsamples/"
fnames = []

# Lists of parameters to use
channel = "t"
mMed_lst= [100, 250, 500, 750, 1000, 1500, 2000]
mDark_lst = [10, 20]
ctau_lst = [1, 100, 1000, 1500, 2000]

genfnames = []
for params in itertools.product(mMed_lst, mDark_lst, ctau_lst):
    genfnames.append(genFileName(params[0], params[1], params[2], channel=channel))
print("Files to be processed")
genfnames

In [ ]:
entry_stop = -1
trigs = l1Trigs

effs_dict_tchan = compute_efficiencies(
    trigs, data_dir, mMed_lst, mDark_lst, ctau_lst, channel,
    mode="L1", 
    entry_stop=entry_stop,
    unprescaled=unprescaled,
)

In [14]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienL1LLP_tchan_effs.pkl", "wb") as f:
    pickle.dump(effs_dict_tchan, f)

In [15]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienL1LLP_tchan_effs.pkl", "rb") as f:
    effs_dict_tchan = pickle.load(f)

In [ ]:
savefig = True 
plot_efficiencies(
    effs_dict_tchan, mMed_lst, ctau_lst, mDark_lst, trigs, 
    plotbest=False, 
    savefig=savefig, 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots",
    outfname="tChanL1LLP_plots", 
    title="EMJ t-channel LLP L1 Trigger Efficiency ", 
    unitymax=True
)

In [ ]:
savefig=True
plot_improvements(
    effs_dict_tchan, mMed_lst, ctau_lst, mDark_lst, trigs, 
    plotbest=True, 
    figsize=(12, 8), 
    outdir="./plots",
    title="EMJ t-channel LLP L1+Best Trigger Efficiency ", 
    savefig=savefig, 
    outfname="tChanL1LLP_improvements"
)

## HLT Triggers
In order to make a fair comparison of efficiencies, we only consider those triggers which are unprescaled.

In [18]:
# prescalesHLT = pd.read_csv("/uscms/home/roycruz/nobackup/EMJ/analysis/prescales/hlt_prescales.csv")
# unprescaledHLT = prescalesHLT["Name"][prescalesHLT["2.0e34 Backup"]==1].to_list()

unprescaledHLT = pd.read_csv("/uscms/home/roycruz/nobackup/EMJ/analysis/prescales/hlt_unprescaled.csv")["UnprescaledHLTTrigs"].to_list()

# Removing version number from those keys that have them
unprescaledHLT = [re.sub(r'_v+$', '', k) for k in unprescaledHLT]

## s-channel

In [ ]:
# Defining parameter valuess
data_dir = "/eos/uscms/store/user/rcruzcan/EMJsamples/"
fnames = []

# Lists of parameters to use
channel = "s"
mMed_lst= [100, 250, 500, 750, 1000, 1500, 2000]
mDark_lst = [10, 20]
ctau_lst = [1, 100, 1000, 1500, 2000]

genfnames = []
for params in itertools.product(mMed_lst, mDark_lst, ctau_lst):
    genfnames.append(genFileName(params[0], params[1], params[2], channel=channel))
print("Files to be processed")
genfnames

In [ ]:
entry_stop = -1

excludedtrigs = [
    'HLT_HcalPhiSym',
    'HLT_HcalNZS',
    'HLT_IsoTrackHE',
    'HLT_PFJet40_GPUvsCPU',
    'HLT_HcalIsolatedbunch'
]

effs_dict_schanHLT = compute_efficiencies(
    hltTrigs, data_dir, mMed_lst, mDark_lst, ctau_lst, channel=channel,
    mode="HLT", 
    entry_stop=entry_stop,
    unprescaled=unprescaledHLT, 
    excludedtrigs=excludedtrigs
)

In [21]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienHLTLLP_schan_effs.pkl", "wb") as f:
    pickle.dump(effs_dict_schanHLT, f)

In [22]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienHLTLLP_schan_effs.pkl", "rb") as f:
    effs_dict_schanHLT = pickle.load(f)

In [ ]:
savefig=False
plot_efficiencies(
    effs_dict_schanHLT, mMed_lst, ctau_lst, mDark_lst, hltTrigs, 
    figsize=(16,6), 
    plotbest=False, 
    title="EMJ s-channel LLP HLT Trigger Efficiency ", 
    savefig=savefig, 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots/", 
    outfname="sChanHLTLLP_plots"
)

In [ ]:
savefig=True
plot_improvements(
    effs_dict_schanHLT, mMed_lst, ctau_lst, mDark_lst, hltTrigs, 
    plotbest=True, 
    figsize=(18, 8), 
    title="EMJ s-channel LLP HLT + Best Trigger Efficiency ", 
    savefig=savefig, 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots/", 
    outfname="sChanHLTLLP_improvements",
)

## t-channel

In [ ]:
# Defining parameter valuess
data_dir = "/eos/uscms/store/user/rcruzcan/EMJsamples/"
fnames = []

# Lists of parameters to use
channel = "t"
mMed_lst= [100, 250, 500, 750, 1000, 1500, 2000]
mDark_lst = [10, 20]
ctau_lst = [1, 100, 1000, 1500, 2000]

genfnames = []
for params in itertools.product(mMed_lst, mDark_lst, ctau_lst):
    genfnames.append(genFileName(params[0], params[1], params[2], channel=channel))
print("Files to be processed")
genfnames

In [ ]:
entry_stop = -1

excludedtrigs = [
    'HLT_HcalPhiSym',
    'HLT_HcalNZS',
    'HLT_IsoTrackHE',
    'HLT_PFJet40_GPUvsCPU',
    'HLT_HcalIsolatedbunch'
]

effs_dict_tchanHLT = compute_efficiencies(
    hltTrigs, data_dir, mMed_lst, mDark_lst, ctau_lst, channel=channel,
    mode="HLT", 
    entry_stop=entry_stop,
    unprescaled=unprescaledHLT, 
    excludedtrigs=excludedtrigs
)

In [27]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienHLTLLP_tchan_effs.pkl", "wb") as f:
    pickle.dump(effs_dict_tchanHLT, f)

In [28]:
with open("/uscms/home/roycruz/nobackup/EMJ/analysis/notebooks/effs/efficienHLTLLP_tchan_effs.pkl", "rb") as f:
    effs_dict_tchanHLT = pickle.load(f)

In [ ]:
savefig=True
plot_efficiencies(
    effs_dict_tchanHLT, mMed_lst, ctau_lst, mDark_lst, hltTrigs, 
    figsize=(16,6), 
    plotbest=False, 
    title="EMJ t-channel LLP HLT Trigger Efficiency ", 
    savefig=savefig, 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots/", 
    outfname="tChanHLTLLP_plots"
)

In [ ]:
savefig=True
plot_improvements(
    effs_dict_tchanHLT, mMed_lst, ctau_lst, mDark_lst, hltTrigs, 
    plotbest=True, 
    figsize=(18, 8), 
    title="EMJ t-channel LLP HLT + Best Trigger Efficiency ", 
    savefig=savefig, 
    outdir="/uscms/home/roycruz/nobackup/EMJ/analysis/plots/", 
    outfname="tChanHLTLLP_improvements",
)

In [ ]:
printBestTrigs(effs_dict_tchanHLT)